In [ ]:
# =============================================================================
# Agents - Autonomous LLM Systems (LangChain 1.0+)
# =============================================================================
"""
=====================================================================
LangChain Agents - LLMs That Take Actions
=====================================================================

What is an Agent?
-----------------
An agent is an LLM that can:
- Decide which tools to use
- Execute tools to gather information
- Reason about results
- Loop until task is complete

Unlike chains (fixed sequence), agents are dynamic - the LLM decides
the execution path based on the task.

Agent Loop:
-----------
    User Query
         │
         ▼
    ┌─────────────────┐
    │   LLM + Tools   │ ◄──────────────────┐
    │   (Reasoning)   │                    │
    └─────────────────┘                    │
         │                                 │
         ▼                                 │
    ┌─────────────────┐                    │
    │  Tool Call?     │                    │
    │  (Yes/No)       │                    │
    └─────────────────┘                    │
         │                                 │
    ┌────┴────┐                            │
    │         │                            │
   Yes        No                           │
    │         │                            │
    ▼         ▼                            │
┌───────┐  ┌───────────┐                   │
│Execute│  │Final      │                   │
│Tool   │  │Answer     │                   │
└───────┘  └───────────┘                   │
    │                                      │
    └──────────────────────────────────────┘

LangChain 1.0+ Agent Methods:
-----------------------------
1. create_agent() - Standard way (recommended)
2. Manual agent loop - Full control
3. LangGraph - Complex multi-step agents

Updated for LangChain 1.0+ (2025-2026)
"""

import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

print("✅ Environment configured for agent examples")

In [ ]:
# =============================================================================
# Step 1: Creating Tools
# =============================================================================
"""
Tools - The Agent's Capabilities
---------------------------------
Tools are functions that agents can call to interact with the world.

The @tool decorator:
- Uses function name as tool name
- Uses docstring as tool description (IMPORTANT for LLM!)
- Generates schema from type hints
"""

from langchain_core.tools import tool
from pydantic import BaseModel, Field
from typing import Optional
import random

print("=" * 60)
print("Step 1: Creating Tools")
print("=" * 60)

# Simple tool - function signature defines schema
@tool
def get_weather(city: str) -> str:
    """
    Get the current weather for a city.
    
    Args:
        city: The city name (e.g., 'London', 'New York')
    
    Returns:
        Current weather conditions
    """
    # Simulated weather data
    conditions = ["sunny", "cloudy", "rainy", "snowy"]
    temp = random.randint(0, 35)
    return f"{city}: {random.choice(conditions)}, {temp}°C"

@tool
def calculate(expression: str) -> str:
    """
    Calculate a mathematical expression.
    
    Args:
        expression: A mathematical expression (e.g., '2 + 2', '100 * 5')
    
    Returns:
        The result of the calculation
    """
    try:
        # Safe evaluation (in production, use a proper math parser)
        result = eval(expression)
        return f"{expression} = {result}"
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def search_knowledge_base(query: str) -> str:
    """
    Search the internal knowledge base for information.
    
    Args:
        query: The search query
    
    Returns:
        Relevant information from the knowledge base
    """
    # Simulated knowledge base
    kb = {
        "langchain": "LangChain is a framework for building LLM applications.",
        "python": "Python is a high-level programming language.",
        "agent": "An agent is an LLM that can use tools to accomplish tasks."
    }
    
    for key, value in kb.items():
        if key in query.lower():
            return value
    return "No relevant information found."

# List of tools
tools = [get_weather, calculate, search_knowledge_base]

print(f"\n🔧 Created {len(tools)} tools:")
for t in tools:
    print(f"   - {t.name}: {t.description[:50]}...")

In [ ]:
# =============================================================================
# Step 2: create_agent() - The Standard Way (LangChain 1.0+)
# =============================================================================
"""
create_agent() Function
-----------------------
The recommended way to create agents in LangChain 1.0+.

Features:
- Simple API
- Built-in tool execution
- Supports middleware
- Checkpointing for persistence
- Streaming support

Note: create_agent() uses LangGraph internally for execution.
"""

from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

print("=" * 60)
print("Step 2: create_agent() - Standard Method")
print("=" * 60)

# Create agent with tools
agent = create_agent(
    model="gpt-4o-mini",  # Model name or model instance
    tools=tools,
    # Optional: Add system prompt
    # system_prompt="You are a helpful assistant..."
)

print("\n✅ Agent created with create_agent()")

# Test the agent
print("\n🤖 Testing agent...")
result = agent.invoke({
    "messages": [HumanMessage(content="What's the weather in Tokyo?")]
})

# Get the final response
final_message = result["messages"][-1]
print(f"\n💬 Agent response: {final_message.content}")

In [ ]:
# =============================================================================
# Step 3: Manual Agent Loop (Pure LangChain)
# =============================================================================
"""
Manual Agent Loop
-----------------
Build an agent from scratch using just LangChain (no LangGraph).
This gives you full control over the execution logic.

Steps:
1. Bind tools to model
2. Send messages to model
3. Check for tool calls
4. Execute tools if needed
5. Loop until no more tool calls
"""

from langchain.chat_models import init_chat_model
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
from typing import List, Dict, Any

print("=" * 60)
print("Step 3: Manual Agent Loop")
print("=" * 60)

# Initialize model
model = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0)

# Bind tools to model
model_with_tools = model.bind_tools(tools)

# Create tool lookup dict
tools_dict = {tool.name: tool for tool in tools}

def execute_tool_calls(tool_calls: List[Dict]) -> List[ToolMessage]:
    """Execute tool calls and return ToolMessages."""
    results = []
    for tc in tool_calls:
        tool_name = tc["name"]
        tool_args = tc["args"]
        tool_id = tc["id"]
        
        print(f"   🔧 Executing: {tool_name}({tool_args})")
        
        if tool_name in tools_dict:
            result = tools_dict[tool_name].invoke(tool_args)
        else:
            result = f"Error: Tool '{tool_name}' not found"
        
        print(f"   📤 Result: {result}")
        results.append(ToolMessage(content=str(result), tool_call_id=tool_id))
    
    return results

def run_agent(user_input: str, max_iterations: int = 5) -> str:
    """
    Run the agent loop manually.
    
    Args:
        user_input: The user's question
        max_iterations: Safety limit on tool execution cycles
    
    Returns:
        The agent's final response
    """
    messages = [
        SystemMessage(content="You are a helpful assistant with access to tools."),
        HumanMessage(content=user_input)
    ]
    
    print(f"\n👤 User: {user_input}")
    
    for i in range(max_iterations):
        print(f"\n🔄 Iteration {i+1}")
        
        # Get model response
        response = model_with_tools.invoke(messages)
        messages.append(response)
        
        # Check if model wants to use tools
        if not response.tool_calls:
            print(f"\n✅ Final answer reached")
            return response.content
        
        # Execute tools and add results
        tool_results = execute_tool_calls(response.tool_calls)
        messages.extend(tool_results)
    
    return "Max iterations reached"

# Test the manual agent
answer = run_agent("What's 25 * 4 and what's the weather in Paris?")
print(f"\n💬 Final Answer: {answer}")

In [ ]:
# =============================================================================
# Step 4: Agent with Conversation Memory
# =============================================================================
"""
Agents with Memory
------------------
Maintain conversation history across turns.
The agent remembers previous questions and answers.
"""

print("=" * 60)
print("Step 4: Agent with Conversation Memory")
print("=" * 60)

class ConversationalAgent:
    """Agent that maintains conversation history."""
    
    def __init__(self, model_with_tools, tools_dict):
        self.model = model_with_tools
        self.tools = tools_dict
        self.history = [
            SystemMessage(content="You are a helpful assistant. Remember our conversation.")
        ]
    
    def chat(self, user_input: str, max_iterations: int = 5) -> str:
        """Process a chat turn with memory."""
        self.history.append(HumanMessage(content=user_input))
        
        for _ in range(max_iterations):
            response = self.model.invoke(self.history)
            self.history.append(response)
            
            if not response.tool_calls:
                return response.content
            
            # Execute tools
            for tc in response.tool_calls:
                if tc["name"] in self.tools:
                    result = self.tools[tc["name"]].invoke(tc["args"])
                else:
                    result = "Tool not found"
                self.history.append(ToolMessage(content=str(result), tool_call_id=tc["id"]))
        
        return "Max iterations reached"
    
    def clear_history(self):
        """Reset conversation history."""
        self.history = [self.history[0]]  # Keep system message

# Create conversational agent
conv_agent = ConversationalAgent(model_with_tools, tools_dict)

# Multi-turn conversation
conversation = [
    "What's the weather in London?",
    "How about New York?",
    "Which city is warmer based on what you told me?"
]

for user_msg in conversation:
    print(f"\n👤 User: {user_msg}")
    response = conv_agent.chat(user_msg)
    print(f"🤖 Agent: {response}")
    print("-" * 40)

In [ ]:
# =============================================================================
# Step 5: Tool with Pydantic Schema
# =============================================================================
"""
Complex Tools with Pydantic
---------------------------
For tools with complex inputs, use Pydantic models.
This provides validation and better documentation.
"""

print("=" * 60)
print("Step 5: Tools with Pydantic Schema")
print("=" * 60)

class FlightSearchInput(BaseModel):
    """Input for flight search."""
    origin: str = Field(..., description="Origin airport code (e.g., 'JFK', 'LAX')")
    destination: str = Field(..., description="Destination airport code")
    date: str = Field(..., description="Travel date in YYYY-MM-DD format")
    passengers: int = Field(default=1, description="Number of passengers", ge=1, le=9)

@tool(args_schema=FlightSearchInput)
def search_flights(origin: str, destination: str, date: str, passengers: int = 1) -> str:
    """
    Search for available flights between two airports.
    
    Returns flight options with prices.
    """
    # Simulated flight search
    prices = [random.randint(200, 800) for _ in range(3)]
    return f"""Flights {origin} → {destination} on {date}:
    1. Morning (8:00 AM): ${prices[0]} per person
    2. Afternoon (2:00 PM): ${prices[1]} per person  
    3. Evening (7:00 PM): ${prices[2]} per person
    Total for {passengers} passenger(s): ${min(prices) * passengers} - ${max(prices) * passengers}"""

# Test the tool
print("\n📋 Tool Schema:")
print(f"   {search_flights.args_schema.schema()}")

print("\n🔧 Direct tool call:")
result = search_flights.invoke({
    "origin": "JFK",
    "destination": "LAX",
    "date": "2025-03-15",
    "passengers": 2
})
print(result)

In [ ]:
# =============================================================================
# Step 6: Agent with Guardrails
# =============================================================================
"""
Adding Guardrails to Agents
---------------------------
Implement safety constraints:
- Input validation
- Tool restrictions
- Output filtering
- Iteration limits
"""

print("=" * 60)
print("Step 6: Agent with Guardrails")
print("=" * 60)

class GuardedAgent:
    """Agent with safety guardrails."""
    
    BLOCKED_TOPICS = ["password", "credit card", "ssn", "hack"]
    MAX_ITERATIONS = 5
    MAX_TOOL_CALLS_PER_TURN = 3
    
    def __init__(self, model_with_tools, tools_dict):
        self.model = model_with_tools
        self.tools = tools_dict
    
    def validate_input(self, user_input: str) -> tuple[bool, str]:
        """Check if input is safe to process."""
        input_lower = user_input.lower()
        for topic in self.BLOCKED_TOPICS:
            if topic in input_lower:
                return False, f"⚠️ Cannot process requests about '{topic}'"
        return True, ""
    
    def validate_tool_calls(self, tool_calls: List) -> List:
        """Limit and validate tool calls."""
        return tool_calls[:self.MAX_TOOL_CALLS_PER_TURN]
    
    def run(self, user_input: str) -> Dict[str, Any]:
        """Run agent with guardrails."""
        # Guardrail 1: Validate input
        is_valid, error_msg = self.validate_input(user_input)
        if not is_valid:
            return {"success": False, "response": error_msg, "tool_calls": 0}
        
        messages = [
            SystemMessage(content="You are a helpful assistant. Be concise."),
            HumanMessage(content=user_input)
        ]
        
        total_tool_calls = 0
        
        # Guardrail 2: Iteration limit
        for iteration in range(self.MAX_ITERATIONS):
            response = self.model.invoke(messages)
            messages.append(response)
            
            if not response.tool_calls:
                return {
                    "success": True,
                    "response": response.content,
                    "tool_calls": total_tool_calls,
                    "iterations": iteration + 1
                }
            
            # Guardrail 3: Limit tool calls per turn
            limited_calls = self.validate_tool_calls(response.tool_calls)
            total_tool_calls += len(limited_calls)
            
            for tc in limited_calls:
                result = self.tools[tc["name"]].invoke(tc["args"]) if tc["name"] in self.tools else "Error"
                messages.append(ToolMessage(content=str(result), tool_call_id=tc["id"]))
        
        return {
            "success": False,
            "response": "Max iterations reached",
            "tool_calls": total_tool_calls
        }

# Test guarded agent
guarded_agent = GuardedAgent(model_with_tools, tools_dict)

# Safe request
print("\n✅ Safe request:")
result = guarded_agent.run("What's the weather in Berlin?")
print(f"   Response: {result['response']}")
print(f"   Tool calls: {result['tool_calls']}")

# Blocked request
print("\n❌ Blocked request:")
result = guarded_agent.run("What's my password?")
print(f"   Response: {result['response']}")

In [ ]:
# =============================================================================
# Step 7: Streaming Agent Responses
# =============================================================================
"""
Streaming Agent Output
----------------------
Stream tokens as the agent generates its response.
Shows both tool calls and final answer in real-time.
"""

print("=" * 60)
print("Step 7: Streaming Agent Responses")
print("=" * 60)

async def stream_agent(user_input: str):
    """Stream agent responses including tool calls."""
    messages = [
        SystemMessage(content="You are a helpful assistant."),
        HumanMessage(content=user_input)
    ]
    
    print(f"\n👤 User: {user_input}")
    print("\n🤖 Agent: ", end="", flush=True)
    
    for _ in range(5):  # Max iterations
        full_response = None
        
        async for chunk in model_with_tools.astream(messages):
            if chunk.content:
                print(chunk.content, end="", flush=True)
            full_response = chunk
        
        if not full_response or not full_response.tool_calls:
            print()  # Newline after streaming
            break
        
        # Handle tool calls
        messages.append(full_response)
        print(f"\n   [Calling tools...]")
        
        for tc in full_response.tool_calls:
            result = tools_dict[tc["name"]].invoke(tc["args"]) if tc["name"] in tools_dict else "Error"
            print(f"   🔧 {tc['name']}: {result}")
            messages.append(ToolMessage(content=str(result), tool_call_id=tc["id"]))
        
        print("\n🤖 Agent: ", end="", flush=True)

# Run streaming agent
await stream_agent("Calculate 100 / 4 and tell me the weather in Tokyo")

In [ ]:
# =============================================================================
# Summary: Agents in LangChain 1.0+
# =============================================================================
"""
=====================================================================
KEY TAKEAWAYS - Agents in LangChain
=====================================================================

1. CREATE TOOLS:
   --------------
   from langchain_core.tools import tool
   
   @tool
   def my_tool(arg: str) -> str:
       '''Tool description (used by LLM)'''
       return result

2. USING create_agent() (RECOMMENDED):
   ------------------------------------
   from langchain.agents import create_agent
   
   agent = create_agent(
       model="gpt-4o-mini",
       tools=[tool1, tool2]
   )
   result = agent.invoke({"messages": [HumanMessage(...)]})

3. MANUAL AGENT LOOP:
   -------------------
   model_with_tools = model.bind_tools(tools)
   
   while True:
       response = model_with_tools.invoke(messages)
       if not response.tool_calls:
           break  # Got final answer
       # Execute tools, add results, continue

4. TOOL EXECUTION:
   ----------------
   for tc in response.tool_calls:
       result = tools_dict[tc["name"]].invoke(tc["args"])
       messages.append(ToolMessage(
           content=result,
           tool_call_id=tc["id"]
       ))

5. GUARDRAILS:
   ------------
   - Validate input before processing
   - Limit tool calls per turn
   - Set max iterations
   - Filter sensitive content

6. BEST PRACTICES:
   ----------------
   - Write clear tool descriptions (LLM reads them)
   - Use temperature=0 for consistent tool calling
   - Implement error handling in tools
   - Add iteration limits (prevent infinite loops)
   - Log tool calls for debugging

Common Imports:
---------------
from langchain.agents import create_agent
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain.chat_models import init_chat_model

=====================================================================
"""

print("=" * 60)
print("Agents Module Complete!")
print("=" * 60)
print("""
Agent Patterns:
---------------
1. create_agent() - Standard, recommended approach
2. Manual loop - Full control, no LangGraph
3. Conversational - With memory
4. Guarded - With safety constraints

For complex multi-agent systems, explore LangGraph!

🎉 Congratulations! You've completed the LangChain tutorial series!
""")